# **Problem Statement**  
## **25. Build a recommendation system using collaborative filtering.**

Build a recommendation system using collaborative filtering to recommend items to users based on historical user–item interactions.

The system should:
- Learn from past user ratings
- Recommend unseen items to users
- Support both brute-force and optimized approaches

### Constraints & Example Inputs/Outputs

### Constraints
- User–Item interaction data is sparse
- Ratings may be explicit (1–5 stars)
- Cold-start problem exists for new users/items
- Dataset size can grow large

### Example Input:
```python
User  Item  Rating
U1    I1    5
U1    I2    3
U2    I1    4
U2    I3    2
U3    I2    4

```

Expected Output:
```python
Recommended items for User U1:
I3, I4

```

### Solution Approach

### Step1: Create User-Item Matrix
- Rows → Users
- Columns → Items
- Values → Ratings

### Step 2: Brute Force Collaborative Filtering
- Compute user–user similarity
- Recommend items liked by similar users
- Uses cosine similarity (manual / sklearn)

### Step 3: Optimized Collaborative Filtering
- Use matrix factorization (SVD)
- Reduce dimensionality
- Faster and scalable

### Step4: Generate Recommendations 
- Predict ratings for unseen items
- Rank items by predicted score

### Solution Code

In [1]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD


In [2]:
# Step2: Create Sample Dataset
data = {
    "user": ["U1", "U1", "U2", "U2", "U3", "U3"],
    "item": ["I1", "I2", "I1", "I3", "I2", "I3"],
    "rating": [5, 3, 4, 2, 4, 5]
}

df = pd.DataFrame(data)
df


,user,item,rating
0,U1,I1,5
1,U1,I2,3
2,U2,I1,4
3,U2,I3,2
4,U3,I2,4
5,U3,I3,5


In [3]:
# Step 3: Create User-Item Matrix
user_item_matrix = df.pivot_table(
    index="user",
    columns="item",
    values="rating"
).fillna(0)

user_item_matrix


item,I1,I2,I3
user,,,
U1,5.0,3.0,0.0
U2,4.0,0.0,2.0
U3,0.0,4.0,5.0


In [4]:
# Approach 1: Brute Force Approach (User-Based CF)
# Step 5: Compute Similarity
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(
    user_similarity,
    index=user_item_matrix.index,
    columns=user_item_matrix.index
)

user_similarity_df


user,U1,U2,U3
user,,,
U1,1.000000,0.766965,0.321403
U2,0.766965,1.000000,0.349215
U3,0.321403,0.349215,1.000000


In [5]:
# Step 5: Recommend Items
def recommend_items_bruteforce(user_id, top_n=2):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]
    
    weighted_scores = np.dot(
        similar_users.values,
        user_item_matrix.loc[similar_users.index]
    )
    
    scores = pd.Series(
        weighted_scores,
        index=user_item_matrix.columns
    )
    
    already_rated = user_item_matrix.loc[user_id] > 0
    scores = scores[~already_rated]
    
    return scores.sort_values(ascending=False).head(top_n)


In [6]:
recommend_items_bruteforce("U1")

item
I3    3.140945
dtype: float64

### Alternative Solution

In [9]:
# Approach 2: Optimized Approach (Matrix Factorization - SVD)
# Step 6: Apply SVD
svd = TruncatedSVD(n_components=2, random_state=42)
latent_matrix = svd.fit_transform(user_item_matrix)


In [10]:
# Step 7: Predict Ratings 
predicted_ratings = np.dot(
    latent_matrix,
    svd.components_
)

predicted_df = pd.DataFrame(
    predicted_ratings,
    index=user_item_matrix.index,
    columns=user_item_matrix.columns
)

predicted_df


item,I1,I2,I3
user,,,
U1,5.263111,1.946748,0.815863
U2,3.617050,1.532973,0.812539
U3,0.019074,3.923645,5.059146


In [11]:
# Step 8: Recommend Items
def recommend_items_svd(user_id, top_n=2):
    user_scores = predicted_df.loc[user_id]
    already_rated = user_item_matrix.loc[user_id] > 0
    user_scores = user_scores[~already_rated]
    return user_scores.sort_values(ascending=False).head(top_n)


In [12]:
recommend_items_svd("U1")

item
I3    0.815863
Name: U1, dtype: float64

### Alternative Approaches

**Brute Force**
- Item-based collaborative filtering
- Pearson correlation similarity

**Optimized**
- ALS (Alternating Least Squares)
- Neural Collaborative Filtering
- Graph-based recommenders
- Hybrid (content + CF)

### Test Case

In [14]:
# Test Case 1: Output Type Check
output = recommend_items_bruteforce("U1")
assert isinstance(output, pd.Series)
print("Test Case 1 Passed")


Test Case 1 Passed


In [19]:
# Test Case 2: No ALready-Rated Items
rated_items = user_item_matrix.loc["U1"]
recommended_items = recommend_items_svd("U1").index

for item in recommended_items:
    assert rated_items[item] == 0

print("Test Case 2 Passed")


Test Case 2 Passed


In [20]:
# Test Case 3: Top-N Constraint
assert len(recommend_items_svd("U2", top_n=1)) == 1
print("Test Case 3 Passed")


Test Case 3 Passed


In [21]:
# Test Case 4: New User Handling
try:
    recommend_items_svd("U100")
except KeyError:
    print("Cold start detected correctly")


Cold start detected correctly


In [22]:
# Test Case 5: Consistency Check
bf_rec = recommend_items_bruteforce("U3")
svd_rec = recommend_items_svd("U3")

assert len(bf_rec) > 0 and len(svd_rec) > 0
print("Test Case 5 Passed")


Test Case 5 Passed


## Complexity Analysis

### Time Complexity
O(1) (constant time)

### Space Complexity
O(1) (constant space)

#### Thank You!!